# SMA Crossover Strategy in backtesting.py

* Buy at market price if the fast SMA is greater than the slow SMA
* If in the market, sell if the fast SMA is smaller than the slow SMA
* Only 1 active operation is allowed in the market

## 1. Reading Data

In [1]:
import pandas as pd
from utils import read_bars as read_bars_tmp

def read_bars(csv_file: str)->pd.DataFrame:
    TIME_BEGIN = pd.to_datetime('2020-05-09T00:00:00.000Z')
    TIME_END = pd.to_datetime('2020-05-15T00:00:00.000Z')
    bars_df = read_bars_tmp(csv_file)
    bars_df = bars_df[(bars_df['timestamp'] >= TIME_BEGIN) & (bars_df['timestamp_end'] < TIME_END)]
    # required by backtesting.py
    bars_df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
    return bars_df

In [2]:
time_bars = read_bars('/data/bars/TimeBar/60000/TimeBar.60000.Binance.Swap.BTC_USDT.csv')

In [3]:
time_bars

,exchange,market_type,pair,bar_type,bar_size,timestamp,timestamp_end,Open,High,Low,...,Volume,volume_sell,volume_buy,volume_quote,volume_quote_sell,volume_quote_buy,count,count_sell,count_buy,vwap
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-05-09 00:00:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-09 00:00:00+00:00,2020-05-09 00:01:00+00:00,9809.629883,9813.250000,9789.969727,...,1008.39600,215.396000,793.000000,9882771.0,2111057.80,9882556.0,1258,462,796,9800.486119
2020-05-09 00:01:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-09 00:01:00+00:00,2020-05-09 00:02:00+00:00,9794.059570,9799.129883,9772.410156,...,1198.93700,529.573000,669.364000,11730592.0,5182406.50,11730062.0,2013,838,1175,9784.160469
2020-05-09 00:02:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-09 00:02:00+00:00,2020-05-09 00:03:00+00:00,9783.639648,9792.879883,9760.000000,...,1730.49200,732.479000,998.013000,16919128.0,7162894.50,16918394.0,2494,1117,1377,9777.062246
2020-05-09 00:03:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-09 00:03:00+00:00,2020-05-09 00:04:00+00:00,9770.530273,9773.419922,9731.860352,...,3147.55600,1281.505000,1866.051000,30687044.0,12495101.00,30685762.0,4130,1836,2294,9749.483091
2020-05-09 00:04:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-09 00:04:00+00:00,2020-05-09 00:05:00+00:00,9760.940430,9797.000000,9755.009766,...,1287.73400,810.601000,477.133000,12586653.0,7923817.00,12585843.0,2205,1350,855,9774.264716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-14 23:54:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-14 23:54:00+00:00,2020-05-14 23:55:00+00:00,9782.389648,9789.000000,9781.709961,...,135.79199,99.256996,36.535000,1328679.8,971229.94,1328580.5,240,137,103,9784.669921
2020-05-14 23:55:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-14 23:55:00+00:00,2020-05-14 23:56:00+00:00,9786.919922,9793.280273,9785.889648,...,124.05800,54.246998,69.811000,1214447.2,531026.30,1214393.0,286,123,163,9789.350143
2020-05-14 23:56:00+00:00,Binance,Swap,BTC_USDT,TimeBar,60000,2020-05-14 23:56:00+00:00,2020-05-14 23:57:00+00:00,9786.280273,9791.879883,9782.000000,...,126.75100,66.317000,60.434002,1240437.4,649066.70,1240371.1,336,170,166,9786.411153


## 2. SMA Crossover Strategy Demo

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    fast = 128
    slow = 512
    
    def init(self):
        # Precompute the two moving averages
        self.sma_fast = self.I(SMA, self.data.Close, self.fast)
        self.sma_slow = self.I(SMA, self.data.Close, self.slow)
    
    def next(self):
        # If sma_fast crosses above sma_slow, close any existing
        # short trades, and buy the asset
        if crossover(self.sma_fast, self.sma_slow):
            self.position.close()
            self.buy()

        # Else, if sma_fast crosses below sma_slow, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()
            self.sell()

/home/frankdai/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
from backtesting import Backtest

bt = Backtest(time_bars, SmaCross, cash=10000.0, commission=0.0004)
stats = bt.run()
stats

Start                     2020-05-09 00:00...
End                       2020-05-14 23:58...
Duration                      5 days 23:58:00
Exposure Time [%]                     87.0104
Equity Final [$]                      10855.6
Equity Peak [$]                         11511
Return [%]                            8.55575
Buy & Hold Return [%]                0.112626
Max. Drawdown [%]                    -14.3379
Avg. Drawdown [%]                    -2.26736
Max. Drawdown Duration        4 days 23:38:00
Avg. Drawdown Duration        0 days 16:09:00
# Trades                                   14
Win Rate [%]                          21.4286
Best Trade [%]                        8.29508
Worst Trade [%]                      -2.98418
Avg. Trade [%]                       0.491042
Max. Trade Duration           1 days 07:08:00
Avg. Trade Duration           0 days 08:58:00
Profit Factor                         1.67665
Expectancy [%]                        2.15202
SQN                               

In [6]:
bt.plot()

## 3. Searching for Optimal Parameters with Backtesting

In [7]:
%%time

stats = bt.optimize(fast=[32, 64, 128],
                    slow=[256, 512, 1024],
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.fast < param.slow)
stats

CPU times: user 350 ms, sys: 336 ms, total: 685 ms
Wall time: 2.27 s


Start                     2020-05-09 00:00...
End                       2020-05-14 23:58...
Duration                      5 days 23:58:00
Exposure Time [%]                     87.6771
Equity Final [$]                      11273.2
Equity Peak [$]                       11860.2
Return [%]                            12.7324
Buy & Hold Return [%]                0.112626
Max. Drawdown [%]                    -7.19234
Avg. Drawdown [%]                   -0.807792
Max. Drawdown Duration        3 days 05:41:00
Avg. Drawdown Duration        0 days 05:12:00
# Trades                                   32
Win Rate [%]                           46.875
Best Trade [%]                        11.8772
Worst Trade [%]                      -1.96867
Avg. Trade [%]                       0.340042
Max. Trade Duration           0 days 10:27:00
Avg. Trade Duration           0 days 03:57:00
Profit Factor                         1.85846
Expectancy [%]                        1.21816
SQN                               

We can look into `stats['_strategy']` to access the Strategy instance and its optimal parameter values (128 and 256).

In [8]:
stats._strategy

<Strategy SmaCross(fast=128,slow=256)>

In [9]:
bt.plot()

## References

* [Backtesting.py Quick Start User Guide](https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html)
* [Moving average crossover - Wikipedia](https://en.wikipedia.org/wiki/Moving_average_crossover)
* [MACD - Wikipedia](https://en.wikipedia.org/wiki/MACD)